In [4]:
!nvidia-smi

Sun Nov 17 08:31:40 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-16GB           On  |   00000000:3B:00.0 Off |                    0 |
| N/A   46C    P0             27W /  250W |       4MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
#Importing necessary libraries

import torch
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

import pandas as pd
import numpy as np
import warnings

from tqdm import tqdm
warnings.filterwarnings("ignore")

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')


/home/hice1/tchavan3/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Using bart large model

# Define training arguments
training_args = TrainingArguments(
    output_dir="/storage/ice1/6/4/tchavan3/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",  # Disable unnecessary logging reports
    
)

In [5]:
#opening the stored model from the path

path = "/storage/ice1/6/4/tchavan3/results-new/checkpoint-18294 - ent"
model = BartForConditionalGeneration.from_pretrained(path)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [7]:
#opening test dataset

test_df = pd.read_csv('/storage/ice1/6/4/tchavan3/technology_test.csv')

In [8]:
index=2
display(test_df[['text','summary']].iloc[index:index+1])

,text,summary
2,Tim Cook thinks he knows how to put $59.7 bill...,Tim Cook thinks he knows how to put $59.7 bill...


In [9]:
test_df['text'][index]

'Tim Cook thinks he knows how to put $59.7 billion to good use\n\nOne of the things that’s keeping Apple’s AAPL market cap from overtaking Exxon Mobil’s XOM — besides Steve Jobs’ health problems and the world’s unquenchable thirst for petroleum products — is the fear that the company will do something stupid with the nearly $60 billion in cash and marketable securities that seems to be burning a hole in Wall Street’s pocket.\n\nThe Street has made it abundantly clear what it thinks Apple should do with that cash: Declare a dividend or launch a stock repurchase program or both — anything to drive up the value of institutional investors’ Apple holdings.\n\nWhat the analysts who work for those institutions fear is that the company will go on a buying spree and start gobbling up the wrong kind of company — like Facebook, Adobe ADBE , Yahoo YHOO , Disney DIS or Sony SNE . Those are some of the acquisition candidates that were floated in the business press last October after Jobs was asked w

In [10]:
test_df['summary'][index]

"Tim Cook thinks he knows how to put $59.7 billion to good use One of the things that's keeping Apple's market cap from overtaking Exxon Mobil's -- besides Steve Jobs' health problems and the world's unquenchable thirst for petroleum products -- is the fear that the company will do something stupid with the nearly $60…"

In [11]:
# we use rouge score as our metric

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []


In [12]:
# Here we generate the summary and evaluate it using ROUGE score

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=2)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

100%|██████████| 5448/5448 [1:03:54<00:00,  1.42it/s]


In [13]:
average_scores_tech = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}


In [14]:
average_scores_tech

{'rouge1': 0.39910534757989524,
 'rouge2': 0.28083376982688085,
 'rougeL': 0.3541314617422803}

In [15]:
test_arch_df=pd.read_csv('/storage/ice1/6/4/tchavan3/architecture_test.csv')
test_arch_df[['text','summary']].head()

,text,summary
0,In what may be the last hurrah for public buil...,Fifth of RIBA's annual architecture plaudits g...
1,"NEW HAVEN, April 27— Vincent Scully did not wa...","Vincent Scully did not want to retire, but Yal..."
2,It's been a good week for women in architectur...,It was a good week for women architects – exce...
3,This piece originally appeared on brunchwork.\...,Start with these three strategies
4,'This is one of those projects that is trying ...,"With its sleek new museum, built by 'winking-e..."


In [16]:
# repeat the process for architecture

rouge_scores_arch=[]
for idx, row in tqdm(test_arch_df.iterrows(), total=test_arch_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores_arch.append(score)

100%|██████████| 419/419 [04:52<00:00,  1.43it/s]


In [17]:
average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores_arch]) / len(rouge_scores_arch) for metric in rouge_scores_arch[0]}
average_scores_arch

{'rouge1': 0.3907207050318286,
 'rouge2': 0.2664953497707454,
 'rougeL': 0.3459093397954607}

In [18]:
test_fd_df=pd.read_csv('/storage/ice1/6/4/tchavan3/food_test.csv')
test_fd_df[['text','summary']].head()

,text,summary
0,When former UMass basketball player Luke Bonne...,The New Hampshire brewery joins up with the Ro...
1,is living the PRISON HIGH LIFE ... at least co...,Lauryn Hill is living the PRISON HIGH LIFE ......
2,The company posted fiscal second-quarter earni...,Darden Restaurants reported quarterly earnings...
3,A shrimp dish at Red Lobster has a calorie cou...,According to the Center for Science in the Pub...
4,Karoline Boehm Goodnick for The Boston Globe\n...,Don’t waste one of the best parts


In [19]:
rouge_scores_fd=[]
for idx, row in tqdm(test_fd_df.iterrows(), total=test_fd_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores_fd.append(score)

100%|██████████| 1482/1482 [17:53<00:00,  1.38it/s]


In [20]:
# repeat the process for food

average_scores_fd = {metric: sum([score[metric].fmeasure for score in rouge_scores_fd]) / len(rouge_scores_fd) for metric in rouge_scores_fd[0]}
average_scores_fd

{'rouge1': 0.3547450392344208,
 'rouge2': 0.24155049171221987,
 'rougeL': 0.3130412674235177}

In [21]:
test_sprt_df=pd.read_csv('/storage/ice1/6/4/tchavan3/sports_test.csv')
test_sprt_df[['text','summary']].head()

,text,summary
0,"BADEN— BADEN, West Germany, Sept. 29 - A Roy a...","BADEN, West Germany, Sept. 29 - A Roy al Canad..."
1,Aaron Rodgers can still remember his close enc...,Aaron Rodgers can still remember his close enc...
2,"Wallace Robinson, the 6-foot-8-inch center who...","Wallace Robinson, the 6-foot-8-inch center who..."
3,Treasurer Joe Hockey has urged first home buye...,Asked whether Sydney’s property prices are out...
4,"On Saturday, the Australian freestyler Mack Ho...",Swimmers at the Rio Games who once served susp...


In [22]:
rouge_scores_sprt=[]
for idx, row in tqdm(test_sprt_df.iterrows(), total=test_sprt_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores_sprt.append(score)

100%|██████████| 6163/6163 [1:13:10<00:00,  1.40it/s]


In [23]:
# repeat the process for sports

average_scores_sprt = {metric: sum([score[metric].fmeasure for score in rouge_scores_sprt]) / len(rouge_scores_sprt) for metric in rouge_scores_sprt[0]}
average_scores_sprt

{'rouge1': 0.3903880163025971,
 'rouge2': 0.2608190000958982,
 'rougeL': 0.33826780989148003}

In [24]:
test_ent_df=pd.read_csv('/storage/ice1/6/4/tchavan3/entertainment_test.csv')
test_ent_df[['text','summary']].head()

,text,summary
0,Adam Sandler and Drew Barrymore paired up pret...,Adam Sandler and Drew Barrymore are back toget...
1,FORTUNE — Vine Alternative Investments is rais...,Vine is latest in recent string of 'income fun...
2,updated 03/23/2015 AT 01:45 PM EDT\n\n•origina...,The new Late Late Show host also names an Emmy...
3,Ridley Scott’s Exodus: Gods and Kings is not e...,"Ridley Scott's new movie ""Exodus: Gods and Kin..."
4,You know that it’s gotten bad when Dan Rather ...,You know that it’s gotten bad when Dan Rather ...


In [ ]:
rouge_scores_ent=[]
for idx, row in tqdm(test_ent_df.iterrows(), total=test_ent_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores_ent.append(score)

100%|█████████▉| 6667/6668 [1:19:55<00:00,  1.63it/s]

In [ ]:
# repeat the process for entertainment

average_scores_ent = {metric: sum([score[metric].fmeasure for score in rouge_scores_ent]) / len(rouge_scores_ent) for metric in rouge_scores_ent[0]}
average_scores_ent

In [26]:
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
bart_model.to(device)

rouge_scores_bart=[]
for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = bart_model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores_bart.append(score)

100%|██████████| 5448/5448 [2:29:21<00:00,  1.64s/it]  


In [27]:
#compute the average scores

average_scores_bart = {metric: sum([score[metric].fmeasure for score in rouge_scores_bart]) / len(rouge_scores_bart) for metric in rouge_scores_bart[0]}
average_scores_bart

{'rouge1': 0.25317686616376717,
 'rouge2': 0.16564610536302368,
 'rougeL': 0.21556226523124777}

In [ ]:
testing_text=test_df['text'][11]
testing_text

In [ ]:
inputs = tokenizer(testing_text, return_tensors="pt", truncation=True, max_length=1024).to(device)
# Generate summary
summary_ids = bart_model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
predicted_summary

In [ ]:
test_df['summary'][11]